In [1]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import requests
import json
import re
from multiprocessing.dummy import Pool
import random
import time
import math
from lxml import etree

In [2]:
#搜索页面
init_search = 'https://coll.jd.com/list.html?sub={}&sort=sort_totalsales15_desc&stock=0&page='

#商品页面
init_itemUrl = 'https://item.jd.com/{}.html'

headers={
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36'
        }

init_commentUrl = 'https://sclub.jd.com/comment/productPageComments.action?productId={}&score=0&sortType=5&page={}&pageSize=10&isShadowSku=0&fold=1'
init_priceUrl = 'https://p.3.cn/prices/mgets?skuIds=J_{}'

target = {'行车记录仪': '23931'}

In [4]:
def getitemUrl(target,maxpage = 1):
    itemList = []
    for i in range(1,maxpage+1):
        url = init_search.format(target)  + str(i)
        res = requests.get(url,headers=headers)
        selector = etree.HTML(res.text)
        itemList.extend(selector.xpath('//div[@id="plist"]/ul/li/div/div[@class="p-name"]/a/@href'))
    return itemList

In [9]:
def getitemidList(target,maxpage = 1):
    itemList = []
    itemidList = []
    for i in range(1,maxpage+1):
        url = init_search.format(target)  + str(i)
        res = requests.get(url,headers=headers)
        selector = etree.HTML(res.text)
        itemList.extend(selector.xpath('//div[@id="plist"]/ul/li/div/div[@class="p-name"]/a/@href'))
        
    print('页面获取完成，共有%d页%d个商品'%(maxpage,len(itemList)))
    i = 1
    for itemUrl in itemList:
        res = requests.get('https:'+itemUrl,headers=headers).text
        selector = etree.HTML(res)
        itemidList.extend(selector.xpath('//div[@id="choose-attrs"]/div//div[@class="dd"]/div/@data-sku')) 
        print('\r' + str(i) + '/' + str(len(itemList)),end = '')
        i += 1
    print('所有分类商品获取完成，共有%d个商品'%(len(itemidList)))
    return itemidList

In [10]:
itemidList = getitemidList(target['行车记录仪'],maxpage=1)

页面获取完成，共有1页60个商品
60/60所有分类商品获取完成，共有226个商品


In [24]:
def getitemInfo(itemid):
    itemUrl = init_itemUrl.format(itemid)
    res = openlink(itemUrl)
    selector = etree.HTML(res.text)

    title = "".join(selector.xpath(
                    '//div[@class="sku-name"]/text()')).replace(' ', '').replace('\r', '').replace('\n', '')

    itemDetail = ";".join(selector.xpath('//div[@class="p-parameter"]/ul[@class="parameter2 p-parameter-list"]/li/text()'))

    content = json.loads(requests.get(init_priceUrl.format(itemid)).text)
    price = content[0]['p']

    comment = json.loads(requests.get(init_commentUrl.format(itemid,0)).text)
    goodRate = comment['productCommentSummary']['goodRate']
    commentCount= comment['productCommentSummary']['commentCountStr']
    itemInfo = pd.DataFrame(dict(itemUrl = [itemUrl],
                                itemid = [itemid],
                                title = [title],
                                itemDetail = [itemDetail],
                                price = [price],
                                goodRate = [goodRate],
                                commentCount = [commentCount]),
                                columns=['itemid',
                                            'itemUrl',
                                            'title',
                                            'itemDetail',
                                            'price' ,
                                            'goodRate',
                                            'commentCount'])
    print('#',end = '')
    return itemInfo
    

In [36]:
def getitemComment(itemid):
    itemUrl = init_itemUrl.format(itemid)
    res = openlink(init_commentUrl.format(itemid,0))
    jc = json.loads(res.text)
    maxPage = jc['maxPage']
    eachComment = []
    for page in range(0,maxPage):
        res = openlink(init_commentUrl.format(itemid,page))
        if len(res.text):
            jc = json.loads(res.text)
            if 'comments' in jc:
                commentlist = jc['comments']
                for comment in commentlist:
                    nickname = comment['nickname']
                    if 'productColor' in comment:
                        productColor  = comment['productColor']
                    else :
                        productColor = None
                    score = comment['score']
                    content = comment['content']
                    creationTime = comment['creationTime']
                    if 'afterUserComment' in comment:
                        afterUserComment = comment['afterUserComment']['hAfterUserComment']['content']
                        afterDays = comment['afterUserComment']['created']
                    else :
                        afterUserComment= None
                        afterDays= None
                    eachComment.append(pd.DataFrame(dict(itemid = [itemid],
                                                          itemUrl = [itemUrl],
                                                          nickname = [nickname],
                                                          productColor = [productColor],
                                                          score = [score],
                                                          content = [content],
                                                          creationTime = [creationTime],
                                                          afterUserComment = [afterUserComment],
                                                          afterDays = [afterDays])))
        else :
            print('被干')
        print('\r' + str(page) + '/' + str(maxPage-1),end = '')
    itemComment = pd.concat(eachComment,ignore_index=True)
    return itemComment

In [37]:
def openlink(link):
    maxTryNum = 10
    for tries in range(0,maxTryNum):
        try:
            res = requests.get(link, headers = headers)
            return res
        except:
            if tries < (maxTryNum - 1):
                print("啊呀")
                continue
            else:
                print("Has tried %d times to access url %s, all failed!", maxNum, link)
                break

## 测试代码

In [29]:
target = 23931
itemidList = getitemidList(target,1)

页面获取完成，共有1页60个商品
60/60所有分类商品获取完成，共有226个商品


### 并行爬取商品信息

In [30]:
pool = Pool(8)
time1 =time.time()
itemInfo = pool.map(getitemInfo,itemidList)
pool.close()
pool.join()
time2 =time.time()
print(str(time2-time1))

##################################################################################################################################################################################################################################25.985722303390503


In [41]:
info = pd.concat(itemInfo,ignore_index=True)

In [48]:
info.to_csv('jd行车记录仪信息'+str(time.strftime("%Y-%m-%d"))+'.csv',encoding='utf-8')

### 并行爬取商品评论

In [38]:
pool = Pool(8)
time1 =time.time()
itemInfo = pool.map(getitemComment,itemidList)
pool.close()
pool.join()
time2 =time.time()
print(str(time2-time1))

62/99被干
68/99
68/99被干
1/99被干
81/99被干
27/44被干
63/99被干
64/99被干
80/99被干
69/99被干
28/44被干
82/99被干
64/99被干
70/99被干
2/99被干
70/99被干
81/99被干
83/99被干被干
29/44
65/99被干
71/99被干
3/99
71/99被干
66/99被干
82/99被干
72/99被干
84/99被干
72/99被干被干
67/99被干
66/99被干
4/99
30/44被干
83/99被干
73/99被干
73/99被干
85/99被干被干
68/99
5/99被干被干
67/99
84/99被干
31/44被干
74/99被干
74/99被干
86/99被干
69/99被干
6/99被干
68/99被干
75/99被干被干
85/99
32/44被干
87/99被干
75/99被干
70/99被干
69/99被干
7/99被干
76/99被干
86/99被干
33/44被干
88/99被干
76/99被干
71/99被干被干
77/99
70/99被干被干
8/99
34/44被干
87/99被干
77/99被干
89/99被干
71/99被干
72/99被干
78/99被干
9/99被干
90/99被干
88/99被干被干

35/44被干
72/99被干
79/99被干
73/99被干
10/99被干
79/99被干
91/99被干被干
36/44
73/99被干
89/99被干
80/99被干
74/99被干
11/99被干
80/99被干
92/99被干
74/99被干
90/99被干
37/44被干
81/99被干
81/99被干
12/99被干被干
75/99
93/99被干
75/99被干
91/99被干
38/44被干
82/99被干
82/99被干
13/99被干
76/99被干
94/99被干
76/99被干被干
92/99
39/44被干
83/99被干
83/99被干
14/99被干
77/99被干
77/99被干
95/99被干
40/44被干
84/99被干
93/99被干
78/99被干
84/99被干
15/99被干
96/99被干
78/99被干
41/44被干
85/99被干
94/99被干
79/99被干
85

2/99被干
10/99被干
4/99被干
19/99被干被干
9/99
34/99被干
17/99被干
17/99被干
3/99被干
11/99被干
5/99被干被干
10/99
20/99被干
35/99被干
12/99被干
18/99被干
4/99被干
6/99被干
18/99被干
11/99被干
21/99被干
36/99被干
19/99被干
5/99被干
13/99被干
7/99被干
19/99被干
12/99被干
22/99被干
37/99被干
20/99被干
14/99被干被干
8/99
6/99被干
13/99被干
20/99被干
23/99被干
38/99被干
15/99被干
21/99被干
7/99被干
9/99被干
14/99被干
21/99被干
24/99被干
39/99被干
22/99被干
16/99被干
8/99被干
10/99被干
15/99被干
22/99被干
25/99被干
40/99被干
23/99被干
17/99被干
11/99被干
9/99被干
26/99被干被干
23/99
41/99被干
16/99被干
12/99被干
24/99被干
18/99被干
10/99被干被干
27/99
24/99被干
42/99被干
17/99被干
13/99被干
25/99被干
19/99被干
11/99被干
28/99被干
18/99被干
43/99被干
25/99被干
14/99被干
20/99被干
26/99被干
12/99被干
29/99被干
19/99被干
44/99被干被干
15/99
26/99被干
21/99被干
27/99被干被干
13/99
30/99被干
45/99被干
20/99被干
27/99被干
16/99被干
22/99被干
28/99被干
14/99被干
21/99被干被干
17/99
31/99被干
28/99被干
46/99被干
29/99被干
23/99被干
15/99被干
22/99被干
32/99被干被干
18/99
29/99被干
47/99被干
30/99被干
24/99被干
16/99被干
23/99被干被干
33/99
30/99被干
19/99被干
48/99被干
25/99被干
17/99被干
31/99被干
49/99被干
20/99被干
34/99被干
31/99被干
26/99被干

41/99被干
42/99被干
53/99被干
38/99被干
47/99被干
53/99被干
70/99被干
42/99被干
54/99被干被干
43/99
48/99被干
39/99被干
71/99被干
54/99被干
55/99被干
43/99被干
40/99被干
44/99被干
72/99被干
0/14被干
55/99被干
56/99被干
44/99被干
41/99被干被干
73/99
50/99被干
56/99被干被干

1/14被干
45/99被干
42/99被干
51/99被干
57/99被干
74/99被干
46/99被干
58/99被干
43/99被干
46/99被干
58/99被干
2/149被干
75/99被干被干
47/99
44/99被干
59/99被干
47/99被干
59/99被干
53/99被干
76/99被干
3/14被干
60/99被干
45/99被干
48/99被干
48/99被干
60/99被干
54/99被干
61/99被干
4/14被干被干

77/99被干
49/99被干
49/99被干
55/99被干
62/99被干
5/14被干
61/99被干
78/99被干
47/99被干
50/99被干
63/99被干
56/99被干
62/99被干
6/14被干
79/99被干
50/99被干
48/99被干
64/99被干
51/99被干
57/99被干
63/99被干
80/99被干
52/99被干
49/99被干
7/14被干被干
65/99
51/99被干
64/99被干
81/99被干
58/99被干
53/99被干
50/99被干
8/14被干
66/99被干
59/99被干
82/99被干
65/99被干
52/99被干
54/99被干
51/99被干
9/14被干
60/99被干
67/99被干
83/99被干
66/99被干
53/99被干
55/99被干
52/99被干
10/14被干被干
84/99
61/99被干
68/99被干
67/99被干
54/99被干
11/14被干
53/99被干
56/99被干
62/99被干
69/99被干
85/99被干
68/99被干
55/99被干
12/14被干
54/99被干
57/99被干
86/99被干被干
63/99
70/99被干
69/99被干
55/

0/99被干
52/99被干
51/99被干
7/38被干
37/99被干被干
65/99
63/99被干
55/99被干
1/99被干
66/99被干被干
52/99
56/99被干
8/38被干
38/99被干
64/99被干
54/99被干
53/99被干
67/99被干被干
2/999
65/99被干
9/38被干
55/99被干
54/99被干
68/99被干
3/99被干
40/99被干
66/99被干
10/38被干
57/99被干被干被干
56/99
55/99
69/99被干
67/99被干
4/99被干
11/38被干
41/99被干
58/99被干
57/99被干
70/99被干
56/99被干
5/99被干
68/99被干
12/38被干
59/99被干
42/99被干
58/99被干
71/99被干
57/99被干
6/99被干
13/38被干
69/99被干被干

43/99被干
59/99被干
72/99被干
58/99被干
14/38被干
7/99被干
70/99被干
44/99被干被干被干
61/99
73/99
60/99被干
59/99被干
15/38被干被干
8/99被干
71/99
45/99被干
62/99被干
61/99被干
74/99被干
60/99被干
16/38被干
72/99被干
9/99被干
46/99被干
75/99被干被干
63/99
62/99被干
61/99被干
17/38被干
73/99被干
10/99被干
76/99被干
47/99被干
63/99被干
64/99被干被干被干
74/99
18/38
62/99被干
11/99被干
48/99被干
77/99被干
64/99被干
65/99被干
19/38被干
75/99被干
12/99被干
63/99被干
78/99被干
49/99被干
66/99被干
65/99被干
20/38被干
76/99被干
13/99被干
64/99被干被干
79/99
50/99被干
66/99被干
14/99被干
77/99被干
67/99被干
21/38被干
65/99被干
80/99被干
51/99被干
67/99被干
22/38被干
15/99被干
78/99被干
68/99被干
66/99被干
68/99被干
81/99被干
52/99被干
23/38被干被干

97/99被干
26/54被干
96/99被干
70/94被干
46/99被干
6/99被干被干被干
28/38

98/99被干
97/99被干
27/54被干
71/94被干
47/99被干
7/99被干
29/38被干
52/99被干
98/99被干
99/99被干
28/54被干
48/99被干
72/94被干
8/99被干
53/99被干
30/38被干
99/99被干
73/94被干
29/54被干
49/99被干
9/99被干
54/99被干
31/38被干
74/94被干
10/99被干
55/99被干
50/99被干
30/54被干
32/38被干
75/94被干
56/99被干被干
11/99
51/99被干
0/99被干
33/38被干
0/99被干
12/99被干
52/99被干
32/54被干
77/94被干
57/99被干
34/38被干
1/99被干
33/54被干
53/99被干
35/38被干
1/99被干
78/94被干
14/99被干被干
34/54
54/99被干
36/38被干被干
35/54
15/99被干
59/99被干
2/99被干
2/99被干
16/99被干
80/94被干
36/54被干
60/99被干
3/99被干
3/99被干
56/99被干
17/99被干
81/94被干
37/54被干
37/38被干
61/99被干
4/99被干
4/99被干
18/99被干
57/99被干
82/94被干
38/54被干
62/99被干被干
38/38
5/99被干
19/99被干
5/99被干
58/99被干
83/94
39/54被干
63/99被干
6/99被干
6/99被干
59/99被干
84/94被干
20/99被干
64/99被干
40/54被干
7/99被干
7/99被干
60/99被干
85/94被干
21/99被干
65/99被干
41/54被干
8/99被干
61/99被干
0/38被干
86/94被干
66/99被干
22/99被干
42/54被干
9/99被干
62/99被干
9/99被干
87/94被干
1/38被干
43/54被干
63/99被干
23/99被干
10/99被干
10/99被干
68/99被干
88/94被干
44/54被干
24/99被干
2/38被干
69/99被干
1

55/56被干被干

5/21被干
17/99被干
79/99被干
15/99被干
85/99被干
63/99被干
56/56被干
7/99被干被干
18/99
80/99被干
6/21被干
86/99被干
16/99被干
64/99被干
81/99被干
19/99被干
7/21被干
8/99被干
87/99被干
17/99被干
65/99被干
82/99被干
20/99被干
8/21被干
9/99被干
88/99被干
18/99被干
66/99被干
83/99被干
9/21被干
21/99被干
10/99被干
89/99被干
67/99被干
10/21被干
19/99被干
84/99被干
0/56被干
11/99被干
68/99被干
90/99被干
85/99被干
11/21被干
20/99被干
23/99被干
1/56被干
91/99被干
69/99被干
24/99被干
12/21被干
21/99被干
13/99被干
86/99被干
2/56被干
13/21被干
70/99被干
22/99被干
87/99被干
14/99被干
93/99被干
3/56被干
71/99被干
14/21被干
23/99被干
88/99被干
15/99被干
94/99被干
4/56被干
72/99被干
15/21被干
24/99被干
89/99被干
16/99被干
5/56被干
73/99被干
16/21被干
25/99被干
95/99被干
25/99被干
90/99被干
17/99被干
6/56被干
17/21被干
74/99被干
96/99被干
26/99被干
7/56被干
26/99被干
91/99被干
18/99被干
18/21被干
75/99被干
97/99被干
27/99被干被干
27/99
8/56被干被干
19/99
92/99被干
76/99被干
19/21被干
98/99被干被干
9/56
28/99被干
93/99被干
28/99被干
20/21被干
20/99被干
77/99被干
99/99被干
29/99被干
10/56被干
29/99被干
94/99被干
21/99被干
78/99被干
30/99被干
11/56被干
21/21被干
95/99被干
22/99被干
31/99被干被干
30/99
12/56被干
96/99被干
79/99被干
32/99被干

53/99被干
19/56被干
12/99被干
19/99被干
1/54被干
54/99被干
20/56被干
2/99被干
46/99被干
20/99被干
20/99被干
21/56被干
55/99被干
2/54被干被干
3/99
21/99被干
47/99被干
56/99被干
22/56被干
3/54被干
15/99被干
21/99被干
22/99被干
4/99被干
48/99被干
23/56被干
57/99被干
4/54被干
16/99被干
22/99被干
23/99被干
5/99被干
58/99被干
24/56被干
5/54被干
49/99被干
17/99被干
23/99被干
24/99被干
6/99被干
59/99被干
50/99被干
6/54被干
25/56被干
18/99被干
24/99被干
25/99被干
7/99被干
60/99被干
51/99被干
19/99被干
26/56被干
7/54被干
25/99被干
26/99被干
52/99被干
8/99被干
61/99被干
20/99被干
8/54被干
26/99被干
27/56被干
27/99被干
53/99被干
9/99被干
62/99被干
21/99被干
9/54被干
28/56被干
27/99被干
28/99被干
54/99被干被干
10/99
22/99被干
63/99被干
29/56被干
10/54被干
28/99被干
29/99被干
23/99被干
55/99被干
64/99被干
11/99被干被干
11/54
29/99被干
30/99被干
56/99被干
65/99被干
24/99被干
12/54被干
12/99被干
30/56被干
30/99被干
31/99被干
57/99被干
66/99被干
25/99被干
13/54被干
13/99被干
31/56被干
32/99被干被干

31/99被干
67/99被干
14/54被干
26/99被干
14/99被干
32/56被干
33/99被干
32/99被干
59/99被干
68/99被干被干被干
27/99
33/56
15/54被干
15/99被干
34/99被干
60/99被干
33/99被干
69/99被干
34/56被干
28/99被干
16/54被干
16/99被干
35/99被干
34/99被干
61/99被干
70/99被

76/99被干
50/99被干
99/99被干
94/99被干
57/99被干
60/99被干被干
77/99
29/99被干
51/99被干
95/99被干
58/99被干
61/99被干
78/99被干
30/99被干
52/99被干
18/33被干
96/99被干
59/99被干
62/99被干
0/99被干
53/99被干
31/99被干
19/33被干
63/99被干
60/99被干
80/99被干
97/99被干
54/99被干被干
64/99
20/33被干
1/99被干
61/99被干
98/99被干
55/99被干
81/99被干
65/99被干
33/99被干
21/33被干
2/99被干
99/99被干被干
82/99
56/99被干被干
34/99
66/99被干
63/99被干
3/99被干
22/33被干
57/99被干
67/99被干
4/99被干
83/99被干
35/99被干被干
23/33
64/99被干
58/99被干
68/99被干
5/99被干
84/99被干
36/99被干
59/99被干
24/33被干被干
6/99
0/999被干
69/99被干
85/99被干
37/99被干
60/99被干
66/99被干
25/33被干
7/99被干
70/99被干
86/99被干
61/99被干
1/993被干
67/99被干
8/99被干
62/99被干
27/33被干
68/99被干
9/99被干
63/99被干
2/99被干
64/99被干
29/33被干
3/99被干
65/99被干
30/33被干
4/99被干
66/99被干
31/33被干
5/99被干
67/99被干
6/99被干
68/99被干
7/99被干
8/99被干
69/99被干
9/99被干
70/99被干
10/99被干被干被干
38/99
69/99
10/99被干被干
71/99
32/33被干
87/99被干被干
71/99
11/99被干
11/99被干
39/99被干
12/99被干
33/33被干
70/99被干
72/99被干
88/99被干
72/99被干
12/99被干
40/99被干
13/99被干
89/99被干被干
13/99
71/99被干
73/99被干
73/99被干
14/99被干
41/99被干
90/99被干
72

67/99被干
54/99被干
89/99被干
3/79被干
78/79被干
4/99被干
68/99被干
18/99被干
35/99被干
55/99被干
90/99被干
79/79被干
4/79被干
5/99被干
19/99被干
69/99被干被干
56/99
36/99被干
91/99被干
5/79被干
6/99被干
20/99被干
57/99被干
70/99被干
92/99被干
6/79被干
37/99被干被干
7/99
21/99被干
71/99被干
58/99被干
93/99被干
7/79被干
0/79被干
8/99被干
22/99被干
94/99被干
72/99被干
59/99被干被干
8/79
39/99被干
9/99被干
1/79被干
23/99被干
73/99被干
60/99被干被干
40/99
9/79被干
10/99被干
96/99被干被干
74/99
24/99被干
2/79被干
61/99被干
11/99被干
97/99被干
10/79被干
42/99被干被干
75/99
25/99被干
3/79被干
62/99被干
12/99被干
98/99被干
11/79被干
43/99被干
76/99被干
26/99被干
63/99被干
4/79被干
13/99被干
12/79被干
99/99被干
44/99被干
77/99被干
64/99被干
27/99被干
5/79被干
14/99被干
13/79被干
45/99被干
78/99被干
65/99被干
28/99被干
6/79被干
15/99被干
14/79被干
46/99被干
79/99被干
29/99被干被干
16/99
66/99被干
7/79被干
15/79被干
47/99被干
30/99被干被干

80/99被干
0/99被干被干
16/79
8/79被干
48/99被干
31/99被干
18/99被干
81/99被干
68/99被干
17/79被干
9/79被干
49/99被干
19/99被干
32/99被干
1/99被干
69/99被干
18/79被干
50/99被干
10/79被干被干
33/99
20/99被干
70/99被干
83/99被干
19/79被干
11/79被干
51/99被干
21/99被干
34/99被干
71/99被干
2/99被干
20/79被干
52/99被干

61/79被干
6/99被干
92/99被干
73/99被干
23/99被干
59/99被干
0/99被干
62/79被干
7/99被干
74/99被干
24/99被干被干
93/99
60/99被干
57/99被干
63/79被干
8/99被干
75/99被干
25/99被干
94/99被干
1/99被干
58/99被干
64/79被干被干
26/99
76/99被干
9/99被干
95/99被干
62/99被干
59/99被干
65/79被干
77/99被干
27/99被干
10/99被干
96/99被干
2/99被干
60/99被干
66/79被干被干
28/99
78/99被干
97/99被干
11/99被干
64/99被干
3/99被干
61/99被干
67/79被干
29/99被干
79/99被干
98/99被干
12/99被干
4/99被干
65/99被干
68/79被干
62/99被干
30/99被干
99/99被干
80/99被干
13/99被干
5/99被干
69/79被干
31/99被干
63/99被干
81/99被干
14/99被干
6/99被干
70/79被干被干
66/99
32/99被干
64/99被干
7/99被干
82/99被干
15/99被干
71/79被干
65/99被干被干
67/99
33/99被干
8/99被干
83/99被干
0/99被干
72/79被干
68/99被干被干被干
66/99

34/99被干
17/99被干
84/99被干
73/79被干
69/99被干
1/999被干被干
35/99
10/99被干
18/99被干
85/99被干
74/79被干
68/99被干
70/99被干
36/99被干
19/99被干
11/99被干
86/99被干
75/79被干
2/99被干
37/99被干
20/99被干
12/99被干
87/99被干
76/79被干
3/99被干
70/99被干
38/99被干被干

13/99被干
21/99被干
77/79被干
4/99被干
71/99被干
39/99被干
14/99被干
89/99被干
22/99被干
78/79被干
71/99被干
5/99被干
72/99被干
40/99被干
90/99被干
15/99被干被干
79/79被干
72/99
23/99被干
73/9

In [43]:
comment = pd.concat(itemInfo,ignore_index=True)

In [47]:
comment.to_csv('jd行车记录仪评论'+str(time.strftime("%Y-%m-%d"))+'.csv',encoding='utf-8')